In [ ]:
# Install required libraries
!pip install tensorflow numpy pillow

In [186]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from tensorflow.keras.preprocessing import image
import numpy as np
# Adding the libraries

In [187]:
# setting constants for batchsize, image height, image width and num of cycle
IMG_HEIGHT = 200
IMG_WIDTH = 400
BATCH_SIZE = 32
EPOCHS = 10

# path for the data source
# Noticed that I took the images https://huggingface.co/datasets/MichalMlodawski/closed-open-eyes, even though they are mostly AI generated but they were really good
DATA_DIR = 'datasource'


In [188]:
# Reading the image from the datasource directory and converting them to given size and make the array of that
def load_images_from_folder(base_path):
    images = []
    labels = []
    for label, folder in enumerate(['closed_eye', 'open_eye']):
        folder_path = os.path.join(base_path, folder)
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join(folder_path, filename)
                img = load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                img_array = img_to_array(img) / 255.0
                images.append(img_array)
                labels.append(label)
    return np.array(images), np.array(labels)

In [189]:
# Loading the images by calling the function, function will return the images and the labels for the images
X, y = load_images_from_folder(DATA_DIR)

In [190]:
# Split data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [191]:
# for extracting the features, I am using 3 layers of conv2D followed by 2x2 maxpool layer to reduce the size
model = models.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),  
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
# after getting the features, I flattened it so I can input it to the dense layer of the model
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
# I used dropout to get better accuracy    
    layers.Dropout(0.5),

# As our problem was binary classification, I used sigmoid function
    layers.Dense(1, activation='sigmoid')
])


In [192]:
# I have used the adam optimizer as it works best in most of the cases
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# model summary gave us the summary of all the layers and num of parameters 9,137,345
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_45 (Conv2D)                   │ (None, 198, 398, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_45 (MaxPooling2D)      │ (None, 99, 199, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_46 (Conv2D)                   │ (None, 97, 197, 64)         │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_46 (MaxPooling2D)      │ (None, 48, 98, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_47 (Conv2D)                   │ (None, 46, 96, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_47 (MaxPooling2D)      │ (None, 23, 48, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_12 (Flatten)                 │ (None, 141312)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 64)                  │       9,044,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,137,345 (34.86 MB)

 Trainable params: 9,137,345 (34.86 MB)

 Non-trainable params: 0 (0.00 B)

In [193]:
# I have used two callback functions so that if my model is overfitting, it will stop training and I also saved the model while training
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('ayesha_bhatti.keras', monitor='val_loss', save_best_only=True)

In [194]:
# Training of model
model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint] 
)


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.5081 - loss: 0.8400 - val_accuracy: 0.5110 - val_loss: 0.6916
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - accuracy: 0.5424 - loss: 0.6902 - val_accuracy: 0.6451 - val_loss: 0.6190
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.6224 - loss: 0.6593 - val_accuracy: 0.7283 - val_loss: 0.5549
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.7280 - loss: 0.5591 - val_accuracy: 0.8347 - val_loss: 0.4645
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.8281 - loss: 0.4219 - val_accuracy: 0.8532 - val_loss: 0.3399
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.8927 - loss: 0.2725 - val_accuracy: 0.8960 - val_loss: 0.2766
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.9357 - loss: 0.1781 - val_accuracy: 0.8451 - val_loss: 0.3651
Epoch 8/10


KeyboardInterrupt: 

In [ ]:
# Saving the model
model.save('ayesha_bhatti.keras')

In [ ]:
# I have some other images of my husband that I have tested on the same model
from tensorflow.keras.preprocessing import image
test_images_folder = 'test_images'

In [ ]:
# preprocessing the image so we can get the predicted results
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

In [ ]:

# Loop through all images in the test folder
for filename in os.listdir(test_images_folder):
    img_path = os.path.join(test_images_folder, filename)
    
# Check if the file is an image (you can add more extensions if needed)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        print(f"Processing image: {filename}")
        
        img_array = preprocess_image(img_path)

# Predict the class
        prediction = model.predict(img_array)
        predicted_class = 'open_eye' if prediction[0][0] > 0.5 else 'closed_eye'

        print(f"Predicted class: {predicted_class}")